# 计步数据解析

In [ ]:
import numpy as np
import pandas as pd
import csv
import os
import struct
from datetime import datetime
from zoneinfo import ZoneInfo
from dataclasses import dataclass, field
from typing import List, Optional,Tuple
from scipy import signal
from scipy.signal import resample
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.dates as mdates


import matplotlib.pyplot as plt
import seaborn as sns
# 设置matplotlib参数
plt.style.use('default')  # 使用默认样式

# 尝试设置中文字体支持(如果失败则使用默认字体)
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
except:
    print("注意: 中文字体设置失败,将使用默认字体")
    pass

from script.data_structures import BaseInfo, ACC_Raw_Data, WearInfo, PPG_Raw_Data, TYHX_PPG_Data
from script.data_types import get_full_data_type
from script.common import get_raw_data, parse_filename, expand_acc_data, expand_ppg_data, process_and_merge_sensor_data
from script.wear_detection_dll import *
from script.motion_recognition_dll import *

In [2]:
def save_acc_data_as_txt(final_merged_df, file_name_without_ext,dictory):
    expanded_acc_df = expand_acc_data(final_merged_df)
    json_shake_path = os.path.join(dictory, r"data/json_shake")
    data_list_path = os.path.join(dictory, r"data/creek2_step_shake_data_list.txt")
    if not os.path.exists( json_shake_path):
        os.makedirs( json_shake_path)

    acc_file_path = os.path.join(json_shake_path, file_name_without_ext + ".txt")
    data_lines = [f"{int(row['x'])} {int(row['y'])} {int(row['z'])}" 
                for _, row in expanded_acc_df[['x', 'y', 'z']].iterrows()]
    data_str = '\n'.join(data_lines)

    output = f"""header:
    {{
    waist_omron_steps: 0,
    collar_omron_steps: 0,
    reference_steps: 0,
    standard_steps: 0
    }}
    contents:
    {{
    {data_str}
    }}"""

    with open(acc_file_path, 'w') as f:
        f.write(output)



    # 将文件名写入 data/creek2_step_shake_data_list.txt
    with open(data_list_path, 'a') as f:
        f.write(file_name_without_ext + '\n') 


In [ ]:



import pyperclip

class InteractiveChart:
    def __init__(self, expanded_acc_df, base_info_df, file_name):
        self.file_name = file_name
        self.process_data(expanded_acc_df, base_info_df)
        self.init_plot()
        self.init_interactive()
        
    def process_data(self, acc_df, step_df):
        # --- 处理加速度 ---
        if not acc_df.empty:
            df = acc_df.sort_values('unix_timestamp').reset_index(drop=True)
            self.acc_times = df['unix_timestamp'].values
            self.acc_data = {
                'x': df['x'].values, 
                'y': df['y'].values, 
                'z': df['z'].values
            }
            self.start_time = self.acc_times[0]
        else:
            self.acc_times = np.array([])
            self.start_time = 0

        # --- 处理步数 ---
        if not step_df.empty:
            df = step_df.sort_values('unix_timestamp').reset_index(drop=True)
            self.step_times = df['unix_timestamp'].values
            self.step_cum = df['step'].values
            self.step_diff = np.diff(self.step_cum, prepend=self.step_cum[0])
            if self.start_time == 0 and len(self.step_times) > 0:
                self.start_time = self.step_times[0]
        else:
            self.step_times = np.array([])

    def init_plot(self):
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(20, 12), sharex=True)
        self.fig.suptitle(self.file_name, fontsize=12)

        # 1. 绘制 Acc
        if len(self.acc_times) > 0:
            self.ax1.plot(self.acc_times, self.acc_data['x'], label='X', c='r', lw=0.5)
            self.ax1.plot(self.acc_times, self.acc_data['y'], label='Y', c='g', lw=0.5)
            self.ax1.plot(self.acc_times, self.acc_data['z'], label='Z', c='b', lw=0.5)
            self.ax1.legend(loc='upper right', fontsize='small')
            self.ax1.set_ylabel('Acceleration')
            self.ax1.grid(True, alpha=0.3)

        # 2. 绘制 Step
        self.ax2_twin = self.ax2.twinx() 
        if len(self.step_times) > 0:
            self.ax2.plot(self.step_times, self.step_diff, '.-', c='purple', lw=0.5, markersize=2, label='Diff')
            self.ax2_twin.plot(self.step_times, self.step_cum, c='orange', lw=1, alpha=0.6, label='Total')
            
            self.ax2.set_ylabel('Step Diff')
            self.ax2_twin.set_ylabel('Cumulative Steps')
            
            h1, l1 = self.ax2.get_legend_handles_labels()
            h2, l2 = self.ax2_twin.get_legend_handles_labels()
            self.ax2.legend(h1+h2, l1+l2, loc='upper left', fontsize='small')
            self.ax2.grid(True, alpha=0.3)

        self.ax2.ticklabel_format(axis='x', style='plain', useOffset=True)
        plt.tight_layout()

    def init_interactive(self):
        # 初始化辅助元素
        self.vl1 = self.ax1.axvline(x=self.start_time, c='k', ls='--', alpha=0.6, visible=False, zorder=999)
        self.vl2 = self.ax2.axvline(x=self.start_time, c='k', ls='--', alpha=0.6, visible=False, zorder=999)
        
        # 初始样式
        self.default_bbox = dict(boxstyle="round", fc="w", alpha=0.9)
        self.annot1 = self.ax1.annotate("", xy=(0,0), xytext=(10,10), textcoords="offset points", 
                                        bbox=self.default_bbox, visible=False, zorder=999)
        self.annot2 = self.ax2.annotate("", xy=(0,0), xytext=(10,10), textcoords="offset points", 
                                        bbox=self.default_bbox, visible=False, zorder=999)

        # 绑定移动事件
        self.fig.canvas.mpl_connect('motion_notify_event', self.on_move)
        # 【新增】绑定点击事件
        self.fig.canvas.mpl_connect('button_press_event', self.on_click)

    def update_annot(self, ax_idx, x, y, text):
        annot = self.annot1 if ax_idx == 1 else self.annot2
        ax = self.ax1 if ax_idx == 1 else self.ax2
        
        annot.xy = (x, y)
        annot.set_text(text)
        annot.set_visible(True)
        
        xlim = ax.get_xlim()
        if x > (xlim[0] + xlim[1]) / 2:
            annot.set_position((-10, 10)); annot.set_ha('right')
        else:
            annot.set_position((10, 10)); annot.set_ha('left')

    def on_move(self, event):
        if not event.inaxes:
            self.vl1.set_visible(False); self.vl2.set_visible(False)
            self.annot1.set_visible(False); self.annot2.set_visible(False)
            self.fig.canvas.draw_idle()
            return
        
        x_mouse = event.xdata
        
        # --- 判定区域: ACC ---
        if event.inaxes == self.ax1 and len(self.acc_times) > 0:
            idx = np.searchsorted(self.acc_times, x_mouse)
            idx = np.clip(idx, 0, len(self.acc_times)-1)
            if idx > 0 and abs(self.acc_times[idx-1] - x_mouse) < abs(self.acc_times[idx] - x_mouse):
                idx -= 1
            
            curr_x = self.acc_times[idx]
            
            self.vl1.set_xdata([curr_x, curr_x])
            self.vl2.set_xdata([curr_x, curr_x])
            self.vl1.set_visible(True); self.vl2.set_visible(True)
            
            txt = (f"T: {int(curr_x)}\n" 
                   f"X: {self.acc_data['x'][idx]:.2f}\n"
                   f"Y: {self.acc_data['y'][idx]:.2f}\n"
                   f"Z: {self.acc_data['z'][idx]:.2f}")
            
            self.update_annot(1, curr_x, self.acc_data['x'][idx], txt)
            self.annot2.set_visible(False)

        # --- 判定区域: STEP ---
        elif (event.inaxes == self.ax2 or event.inaxes == self.ax2_twin) and len(self.step_times) > 0:
            idx = np.searchsorted(self.step_times, x_mouse)
            idx = np.clip(idx, 0, len(self.step_times)-1)
            if idx > 0 and abs(self.step_times[idx-1] - x_mouse) < abs(self.step_times[idx] - x_mouse):
                idx -= 1
            
            curr_x = self.step_times[idx]
            
            self.vl1.set_xdata([curr_x, curr_x])
            self.vl2.set_xdata([curr_x, curr_x])
            self.vl1.set_visible(True); self.vl2.set_visible(True)
            
            txt = (f"T: {int(curr_x)}\n"
                   f"Diff: {self.step_diff[idx]}\n"
                   f"Total: {self.step_cum[idx]}")
            
            self.update_annot(2, curr_x, self.step_diff[idx], txt)
            self.annot1.set_visible(False)
            
        self.fig.canvas.draw_idle()

    # ================= 【新增】点击复制逻辑 =================
    def on_click(self, event):
        # 1. 仅响应左键且在坐标轴内
        if event.button != 1 or not event.inaxes:
            return

        # 获取点击处的时间戳
        click_time = event.xdata
        
        # --- 查找最近的 Acc 数据 ---
        acc_str = "N/A\tN/A\tN/A"
        if len(self.acc_times) > 0:
            idx_acc = np.searchsorted(self.acc_times, click_time)
            idx_acc = np.clip(idx_acc, 0, len(self.acc_times)-1)
            # 简单的最近邻校正
            if idx_acc > 0 and abs(self.acc_times[idx_acc-1] - click_time) < abs(self.acc_times[idx_acc] - click_time):
                idx_acc -= 1
            
            # 只有当时间误差在可接受范围内(例如 1秒内)才视为有效,否则可能点击了空白区
            # 这里简单处理,直接获取
            ax = self.acc_data['x'][idx_acc]
            ay = self.acc_data['y'][idx_acc]
            az = self.acc_data['z'][idx_acc]
            acc_str = f"{ax:.2f}\t{ay:.2f}\t{az:.2f}"

        # --- 查找最近的 Step 数据 ---
        step_str = "N/A\tN/A"
        if len(self.step_times) > 0:
            idx_step = np.searchsorted(self.step_times, click_time)
            idx_step = np.clip(idx_step, 0, len(self.step_times)-1)
            if idx_step > 0 and abs(self.step_times[idx_step-1] - click_time) < abs(self.step_times[idx_step] - click_time):
                idx_step -= 1
            
            diff = self.step_diff[idx_step]
            total = self.step_cum[idx_step]
            step_str = f"{diff}\t{total}"

        # --- 组合成 Excel 友好格式 (Time, AccX, AccY, AccZ, StepDiff, StepTotal) ---
        # 这一行粘贴到 Excel 会自动分成 6 列
        # clipboard_text = f"{int(click_time)}\t{acc_str}\t{step_str}"
        clipboard_text = f"{int(click_time)}"
        # 为了方便阅读,控制台打印带标题的竖排版,但剪切板里是横排版
        console_msg = (
            f"T: {int(click_time)}\n"
            f"ACC: {acc_str.replace(chr(9), ', ')}\n"
            f"STEP: {step_str.replace(chr(9), ', ')}"
        )

        # --- 复制与反馈 ---
      
        try:
            pyperclip.copy(clipboard_text)
            print(f"✅ 已复制全量数据 (Excel格式):\n{console_msg}\n" + "-"*40)
        except Exception as e:
            print(f"❌ 复制失败: {e}")
     

        # --- 视觉闪烁反馈 ---
        # 无论点哪个图,两个标签都闪烁一下,表示数据已关联提取
        targets = []
        if self.annot1.get_visible(): targets.append(self.annot1)
        if self.annot2.get_visible(): targets.append(self.annot2)
        
        for t in targets:
            t.get_bbox_patch().set_facecolor('#98FB98') # PaleGreen
            
        def restore_color():
            for t in targets:
                t.get_bbox_patch().set_facecolor('w')
            self.fig.canvas.draw_idle()
        
        timer = self.fig.canvas.new_timer(interval=200)
        timer.add_callback(restore_color)
        timer.start()



In [ ]:
def plot_acc_and_step_data(expanded_acc_df, base_info_df, directory, file_name_without_ext, is_show=False):
    """
    绘制加速度数据和计步数据
    
    参数:
        expanded_acc_df: 展开的加速度数据 DataFrame (包含 x, y, z, unix_timestamp 列)
        base_info_df: 基础信息 DataFrame (包含 unix_timestamp, step 列)
        directory: 保存目录
        file_name_without_ext: 文件名(不含扩展名)
        is_show: 是否显示图表(默认为 False, 保存到文件)
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12), sharex=True)
    
    # 添加文件名作为整个图的标题
    fig.suptitle(file_name_without_ext, fontsize=14, fontweight='bold', y=0.995)
    

    # ax1: 绘制展开的 acc 数据 (x, y, z)
    if not expanded_acc_df.empty and all(col in expanded_acc_df.columns for col in ['x', 'y', 'z']):
        Vector_Sum = np.sqrt(expanded_acc_df['x']**2 + expanded_acc_df['y']**2 + expanded_acc_df['z']**2)
        ax1.plot(expanded_acc_df['unix_timestamp'],expanded_acc_df['x'], label='ACC X', color='red', linewidth=0.5)
        ax1.plot(expanded_acc_df['unix_timestamp'],expanded_acc_df['y'], label='ACC Y', color='green', linewidth=0.5)
        ax1.plot(expanded_acc_df['unix_timestamp'],expanded_acc_df['z'], label='ACC Z', color='blue', linewidth=0.5)
        ax1.plot(expanded_acc_df['unix_timestamp'],Vector_Sum, label='ACC Vector Sum', color='black', linewidth=1.0, alpha=0.7)
        ax1.set_ylabel('Acceleration')
        ax1.set_title(f'Acceleration Data - {len(expanded_acc_df)} samples')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
    
    # ax2: 绘制 step 和 step 的差分
    if not base_info_df.empty and 'step' in base_info_df.columns:
        # 计算步数差分
        step_values = base_info_df['step'].values
        step_diff = np.diff(step_values, prepend=step_values[0])
        
        # 获取起始、结束和总步数
        step_start = base_info_df['step'].iloc[0]
        step_end = base_info_df['step'].iloc[-1]
        step_total = step_end - step_start
        
        # 绘制步数差分(每秒步数)
        # 使用 'o-' 绘制点+线的效果
        ax2.plot(base_info_df['unix_timestamp'], step_diff, '.-', label='Steps/sec', color='purple', 
                 markersize=2, linewidth=0.8, alpha=0.7)
        ax2_twin = ax2.twinx()
        ax2_twin.plot(base_info_df['unix_timestamp'], base_info_df['step'], label='Cumulative Steps', color='orange', linewidth=1.0, alpha=0.7)
        ax2_twin.set_ylabel('Cumulative Steps')
        ax2.set_xlabel('Time (Unix Timestamp)' if 'unix_timestamp' in base_info_df.columns else 'Sample Index')
        ax2.set_ylabel('Steps per Second')
        ax2.set_title(f'Step Data - start:{step_start}, stop:{step_end}, total:{step_total} Sport time:{ base_info_df['unix_timestamp'].max() - base_info_df['unix_timestamp'].min() } sec')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if is_show:
        plt.show()
    else:
        save_fig_name = os.path.join(directory, f"{file_name_without_ext}_acc_step.png")
        plt.savefig(save_fig_name, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"图表已保存: {save_fig_name}")


In [6]:

# directory = r'rawdata/Motion/0108/'

# all_files = os.listdir(directory)

# # 过滤出 .rawdata 文件
# rawdata_files = [f for f in all_files if f.endswith('.rawdata')]
# rawdata_files

In [ ]:
# %matplotlib qt 
%matplotlib inline


# StepCount_Walk_ztf_20260114_163929_5dd7          start:2235 stop:2755   diff: 520
# StepCount_umbrella_ztf_20260114_164552_5dd7      start:2755 stop:3223   diff: 468 
# StepCount_Bag_ztf_20260114_165717_5dd7           start:3710 stop:4152   diff: 442 
# StepCount_Pocket_ztf_20260114_170209_5dd7        start:4152 stop:4642   diff: 490 
# StepCount_Phone_ztf_20260114_170733_5dd7         start:4642 stop:5102   diff: 460
# StepCount_Run_ztf_20260114_171222_5dd7           start:5102 stop:5625   diff: 523 

directory = r'G:\PyScript\rawdata\Step\CWR11G_bug'
file_name = "StepCount_Run_ztf_20260114_171222_5dd7.rawdata"



file_path = os.path.join(directory, file_name)
real_path = os.path.realpath(file_path)
file_name = os.path.basename(file_path)
file_name_without_ext, ext = os.path.splitext(file_name)
directory = os.path.dirname(real_path)
save_dir = os.path.join(directory, "data")
os.makedirs(save_dir, exist_ok=True)
print(f"real_path: {real_path} \n{directory} {file_name_without_ext} {ext}")


result = get_raw_data(real_path)


data_type_dict = result.get('data_types', {})
base_info_df = result.get('base_info_df')
tyhx_df = result.get('TYHX_data_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_agc_data_df = result.get('TYHX_agc_data_df', pd.DataFrame(columns=['unix_timestamp']))

wear_info_df = result.get('wear_info_df', pd.DataFrame(columns=['unix_timestamp']))
motion_recognition_df = result.get('motion_recognition_df')

acc_df =  result.get('acc_df', pd.DataFrame(columns=["unix_timestamp","arr_size","acc_data"]))



# sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
# readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
# print(f"文件 {file_path} 解析完成,包含数据类型: ")
# print(f"{"\n".join(readable_types)}")

acc_df = acc_df[["unix_timestamp","arr_size","acc_data"]].copy() 
df = base_info_df[['unix_timestamp','activity_level', 'heart_rate', 'step','step_freq','wear_flag',]]  # type: ignore

step_diff =  df['step'].iloc[-1] - df['step'].iloc[0]
step_diff2 = df['step'].max() - df['step'].min()
print(f"step start:{ df['step'].iloc[0]} stop:{df['step'].iloc[-1] }   diff: {step_diff }  step2: {step_diff2}   ")

expanded_acc_df = expand_acc_data(acc_df)
# plot_acc_and_step_data(expanded_acc_df, df, directory, file_name_without_ext, is_show=False)

real_path: G:\PyScript\rawdata\Step\CWR11G_bug\StepCount_Run_ztf_20260114_171222_5dd7.rawdata 
G:\PyScript\rawdata\Step\CWR11G_bug StepCount_Run_ztf_20260114_171222_5dd7 .rawdata
step start:5102 stop:5625   diff: 523  step2: 523   


In [8]:
chart = InteractiveChart(expanded_acc_df, base_info_df, file_name_without_ext)
plt.show()

In [13]:
%matplotlib qt 

plot_acc_and_step_data(expanded_acc_df, df, directory, file_name_without_ext, is_show=True)

In [ ]:
%matplotlib qt 
# ================= 实例化运行 =================


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1. 准备数据
x = np.linspace(0, 10, 100)
y = np.sin(x) * np.exp(-x/5)

# 2. 创建画布和基本图形
fig, ax = plt.subplots(figsize=(8, 6))
line, = ax.plot(x, y, label='Data', color='blue')
ax.set_title("移动鼠标查看十字线与数值吸附")
ax.grid(True, alpha=0.3)

# 3. 初始化 十字线 和 标签 (初始状态)
# 垂直线 (VLine) 和 水平线 (HLine)
vl = ax.axvline(x=0, color='red', linestyle='--', alpha=0.6, visible=False)
hl = ax.axhline(y=0, color='red', linestyle='--', alpha=0.6, visible=False)

# 数据标签 (使用 annotate 方便设置偏移)
# xytext=(10, 10) 表示标签在点的右上方偏移 10 像素
label = ax.annotate(
    '', 
    xy=(0, 0), 
    xytext=(10, 10), 
    textcoords='offset points',
    bbox=dict(boxstyle="round", fc="w", alpha=0.8),
    arrowprops=dict(arrowstyle="->")
)
label.set_visible(False)

# 4. 定义鼠标移动的回调函数
def on_mouse_move(event):
    # 如果鼠标不在绘图区域内,直接返回
    if not event.inaxes:
        vl.set_visible(False)
        hl.set_visible(False)
        label.set_visible(False)
        fig.canvas.draw_idle()
        return

    # 获取当前鼠标的 x 坐标
    x_mouse = event.xdata
    
    # --- 核心吸附逻辑 ---
    # 计算数据中所有 x 与鼠标 x 的距离,找到最小值的索引
    # 这是 "最近 X 轴吸附",适用于折线图
    idx = (np.abs(x - x_mouse)).argmin()
    
    # 获取吸附后的具体 x, y 值
    x_snap = x[idx]
    y_snap = y[idx]

    # --- 更新绘图元素 ---
    
    # 1. 更新十字线位置
    # axvline/axhline 需要设置两个点的 x 或 y 数据
    vl.set_xdata([x_snap, x_snap])
    hl.set_ydata([y_snap, y_snap])
    vl.set_visible(True)
    hl.set_visible(True)

    # 2. 更新标签位置和文本
    label.xy = (x_snap, y_snap)
    label.set_text(f'x: {x_snap:.2f}\ny: {y_snap:.2f}')
    label.set_visible(True)

    # 3. 刷新画布 (使用 draw_idle 优化性能)
    fig.canvas.draw_idle()

# 5. 绑定事件
# 'motion_notify_event' 表示鼠标移动事件
fig.canvas.mpl_connect('motion_notify_event', on_mouse_move)

plt.show()

In [10]:

directory = r'G:\python\rawdata_motion\data2'

# G:\python\rawdata_motion\Typing
directory = r'G:\python\rawdata_motion\Typing'



directory = r'rawdata/Motion/251215'


# 获取目录下的所有文件
all_files = os.listdir(directory)

# 过滤出 .rawdata 文件
rawdata_files = [f for f in all_files if f.endswith('.rawdata')]

for f in rawdata_files:
    file_path = os.path.join(directory, f)

    real_path = os.path.realpath(file_path)
    file_name = os.path.basename(file_path)
    file_name_without_ext, ext = os.path.splitext(file_name)
    directory = os.path.dirname(real_path)
    info =  parse_filename(file_name)
    mac = info.get('mac','0000').lower()


    result = get_raw_data(real_path)

    data_type_dict = result.get('data_types', {})
    sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
    readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
    print(f"文件 {file_path} 解析完成,包含数据类型: ")
    print(f"{"\n".join(readable_types)}")

    motion_recognition_df = result.get('motion_recognition_df')
    base_info_df = result.get('base_info_df')
    acc_df = result.get('acc_df')
    if acc_df is None or acc_df.empty:
        print(f"文件 {file_path} 中没有加速度数据,跳过处理。")
        continue
    acc_df = acc_df[["unix_timestamp","arr_size","acc_data"]].copy() # type: ignore
    df = base_info_df[['unix_timestamp','activity_level', 'heart_rate', 'step','step_freq','wear_flag',]]  # type: ignore

    step_diff =  df['step'].iloc[-1] - df['step'].iloc[0]
    step_diff2 = df['step'].max() - df['step'].min()
    print(f"step start:{ df['step'].iloc[0]} stop:{df['step'].iloc[-1] }   diff: {step_diff }  step2: {step_diff2}   ")
    # 调用函数处理和合并 DataFrame
    final_merged_df = process_and_merge_dataframes(df, motion_recognition_df, acc_df)
    Motiondll_with_dataframe(final_merged_df, directory, file_name_without_ext)

文件 rawdata/Motion/251215\HeartRate_IndoorRun_dlh_20251216_170427_5c05.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 614 
15  RAW_PPG_GREEN         原始PPG绿光                count: 642 
17  RAW_PPG_IR            原始PPG红外                count: 642 
18  RAW_ACC               原始加速度                  count: 1284 
21  RAW_ACC_TO_PPG        原始加速度到PPG              count: 1284 
33  AMBIENT_INFO          环境信息                   count: 642 
69  MOTION_INFO           运动识别                   count: 596 
71  WEAR_INFO             佩戴信息                   count: 642 
82  TYHX_AGC_DATA         TYHX AGC数据             count: 642 
step start:7072 stop:8383   diff: 1311  step2: 1311   
日志已保存至: G:\PyScript\rawdata\Motion\251215\HeartRate_IndoorRun_dlh_20251216_170427_5c05_log.txt
文件 G:\PyScript\rawdata\Motion\251215\HeartRate_IndoorRun_dlh_20251216_170427_e61b.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 613 
15  RAW_PPG_GREEN         原始PPG绿光         